<a href="https://colab.research.google.com/github/msteknoadam/mss-dl-challange/blob/master/MSS_DL_Challange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/msteknoadam/mss-dl-challange/raw/master/mss-deep-learning-challenge_dataset.tar.gz

In [11]:
import tarfile
  
# open file
file = tarfile.open('mss-deep-learning-challenge_dataset.tar.gz')
  
# extracting file
file.extractall('.')
  
file.close()

In [13]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# set the directory path for the dataset
dataset_dir = '/content/mss_dataset'

# create a list of all the image paths and their corresponding labels
image_paths = []
labels = []

for category in os.listdir(dataset_dir):
    category_dir = os.path.join(dataset_dir, category)
    for file_name in os.listdir(category_dir):
        file_path = os.path.join(category_dir, file_name)
        image_paths.append(file_path)
        labels.append(category)

# create a Pandas dataframe with the image paths and labels
data = {'image_path': image_paths, 'label': labels}
df = pd.DataFrame(data)

# set the hyperparameters
batch_size = 32
epochs = 10

In [ ]:
print(df)

In [15]:
# create an ImageDataGenerator object for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20
)

# create an ImageDataGenerator object for validation data
valid_datagen = ImageDataGenerator(rescale=1./255)

# split the dataframe into training and validation dataframes
train_df = df.sample(frac=0.8, random_state=42)
valid_df = df.drop(train_df.index)

# create a data generator for the training data
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# create a data generator for the validation data
valid_generator = valid_datagen.flow_from_dataframe(
    valid_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 10637 validated image filenames belonging to 11 classes.
Found 2659 validated image filenames belonging to 11 classes.


In [16]:
# load the VGG16 model
base_model = tf.keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# create a new model on top of the base model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(11, activation='softmax')(x)
model = tf.keras.Model(inputs, x)

# compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=valid_generator
)

Epoch 1/10
  2/333 [..............................] - ETA: 1:45:49 - loss: 3.1209 - accuracy: 0.1250

In [ ]:
# create a data generator for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# create a list of all the image paths and their corresponding labels
image_paths = []
labels = []

for category in os.listdir(dataset_dir):
    category_dir = os.path.join(dataset_dir, category)
    for file_name in os.listdir(category_dir):
        file_path = os.path.join(category_dir, file_name)
        image_paths.append(file_path)
        labels.append(category)

# create a Pandas dataframe with the image paths and labels
data = {'image_path': image_paths, 'label': labels}
df = pd.DataFrame(data)

# create a data generator for the test data
test_generator = test_datagen.flow_from_dataframe(
    df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
# evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)

print('Test accuracy:', test_acc)